# What I Learned From A Titanic Solution

https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy/notebook

### Trying to fill in incomplete data

When you are trying to fill in incomplete data there are 2 things that you can do

1. If it is categorical, you could just take the mode
2. If it is quatatative, you could just take the median

### Figuring out if you columns have NULL values

A sneaky good way of figuring out if your columns have null data would just be to do something like

data.isnull().sum()

### Creating bins for your data

There is 2 easy ways to create bins for your data. 

1. cut(): What this will do is cut your variables into bins of equal size

    data["AgeBin"] = pd.cut(data["Age"].astype(int), 5)
    
    
2. qcut(): What this will do is cut your variable into bins, where there is the name number of variables in each bin

   data["AgeBin"] = pd.qcut(data["Age"].astype(int), 5)

### Getting value counts

data["ValueCounts"].value_counts() 

This will return a object that contains the counts of each unique object. 

In [ ]:
stat_min = 10
title_names = (data1['Title'].value_counts() < stat_min) #this will create a true false series with title name as index
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(data1['Title'].value_counts())

### Sklearn LabelEncoder()

The sklearn library has a label encoder that can be used to encode your categorical variables. Will encode labels with value between 0 and n_classes-1

In [ ]:
label = LabelEncoder()
for dataset in data_cleaner:
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])

### Sklean ModelSelection()

This can be really useful for spliting your data between training and test data.

It takes a parameter called test_size, that is an integer between 0 and 1. This integer is the fraction of the data that will be allocated to the test dataset. 

In [ ]:
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(data1[data1_x_calc], data1[Target], random_state = 0)

### Matplotlib subplots

This with return a figure and axis which can be usefull for graphing multiple things. (numrows, numcols)

In [ ]:
fig, axes = plt.subplot(2, 2)

axes[0, 0].scatter(x, y)
axes[0, 1].plot(x2, y2)

### Seaborn

Seaborn is good if you are doing multi-variable comparison. The FacetGrid function seems ultra usefull.

### Various kinds of plots that I think are pretty useful

__Box Plot__: Good to show the distribution of the data. It will show the median and the box will span the upper and lower quantile range.

__Histogram__: These are used for ploting the fequency of some scores appearing in a continuous data set that has been divided into certain classes. Usually, there will be no spaces between the bars signaling the continuity of the data. 

__Bar Plot__: This is similar to histogram but for categorical variables, this difference can be seen because there are spaces between the bars in this plot. 

__Point Plot__: Seems like it will just show the median of a value on a point graph.

__Scatter Plot__: We all know what a scatter plot is. 

__Violin Plot__: These are decent at showing the probablility density at various values. 

__Seaborn FacetGrids__: You can define different variables in your columns and rows and these values will be selected and graphed. 

__KDE Plot__: This will give you a smoothed probablity density.

__Pair Plot__: When you are just comparing two variables. This might just be a generalized term.

__Seaborn.distplot__: Combination of a histogram and a kde plot.

### Correlation Heatmaps

This seems like it super usefull trying to figure out the relationship between all your variables. When doing this be careful that it will only graph values that are numeric. So you can use LabelEncoder

In [ ]:
#correlation heatmap of dataset
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(not_submit_data)

### Kinds of ML and loose definition

1. Supervised
2. Unsupervised
3. Reinforced (Hybrid of the last 2)

Machine Learning (ML), as the name suggests is teaching the machine how to think and not what to think.

### No Free Lunch Theorum

There is no 'better' machine learning algorithm that works in all scenario, they all work differently in different situations.

Although, we are fairly certain that more data always beats better algorithms. 

### An alternative way to search through machine learning algorithms

In [ ]:
#Machine Learning Algorithm (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]

#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = not_submit_data["Survived"]

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, data1[data1_x_bin], data1["Survived"], cv  = cv_split)
    
    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(data1[data1_x_bin], data1[Target])
    MLA_predict[MLA_name] = alg.predict(data1[data1_x_bin])
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare
#MLA_predict

### Classification vs Regression Algorithms

We can generalize that a continuous target variable requires a regression algorithm and a discrete target variable requires a classification algorithm. Since our problem is wether a passenger survived or not, this is a discrete target variable thus we will have to use a classification algorithm.

### Coinflip (model baselines)

If you know nothing about your model and it's classification then if you just random pick a state and guess that for every data point you would have a (1/(number_of_states)) chance of getting that correct. Always compare your model against that.

Then compare it when you have more information about your data set. So let's say you have a little information about that dataset, that information being the most popular outcome. If you just guess regardless of the data that this was the outcome, you may want to compare your model against that.

### Pivot Table

I think this is similar to a groupby table.

### Confusion Matrix

This is a table that is often used to describe the performance of a classification model on a set of test data for which the true values are known. 

### DT RFE

Recursive feature elimination, not 100% sure what that is but going to figure that out later.

### Counter in Python

A Counter in python is a closely related to the Dict. It will keep the value you are counting as the key and the count as the value. 

### Python Extend

You can add a list of elements to another list easily

In [1]:
x = [1, 2, 3]
x.extend([4, 5])
print (x)

[1, 2, 3, 4, 5]


Skew in your data is bad

Removing outliers is a very good idea

Figure out wtf a factorplot is